In [0]:
from scipy import spatial
from google.colab import files
import pandas as pd
import numpy as np

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 19.6MB/s 


In [0]:
masterTable = pd.read_csv('pre-processed-mastertable2.csv')

In [0]:
#Inputs for model
lat = 28.560323
long = 77.1617281
time = '5:30'

latLong = []
for i,row in masterTable.iterrows():
  latLong += [(row[5],row[6])]
  
tree = spatial.KDTree(latLong)
dist = tree.query([(lat,long)])[0][0]
index = tree.query([(lat,long)])[1][0]

PoliceStations = masterTable['PoliceStations'][index]
Population = masterTable['c-population'][index]
Traffic = masterTable['c-traffic'][index]


def convertTimetoCategoricalValue(time):
	hour = time.split(':')[0]
	hour = int(hour)
	if hour >= 5 and hour < 12:
		return 1
	elif hour >= 12 and hour < 16:
		return 2
	elif hour >= 16 and hour < 21:
		return 3
	else:
		return 4

Time = convertTimetoCategoricalValue(time)

In [0]:
gridM = np.load('murderModel' + '.npy')
gridR = np.load('rapeModel' + '.npy')
gridDV = np.load('domesticModel' + '.npy')
gridT = np.load('theftModel' + '.npy')
gridF = np.load('fraudModel' + '.npy')
gridD = np.load('drugModel' + '.npy')


def getIntensity(lat,lng):
	minLat = 28.4752049
	maxLat = 28.8059341
	minLong =  76.9504542
	maxLong = 77.4538
	if lat < minLat or lat > maxLat or lng < minLong or lng > maxLong:
		return 0.0
	x = (lat - minLat)/(maxLat - minLat) * len(gridM)
	y = (lng - minLong)/(maxLong - minLong) * len(gridM)
	intensity = dict()
	average = 0
	count = 0
	flag = 1
	intensity['Murder'] = gridM[int(round(x))][int(round(y))] 
	if intensity['Murder'] != 0:
		average += intensity['Murder']
		count += 1
		flag = 0
	intensity['Rape'] = gridR[int(round(x))][int(round(y))]
	if intensity['Rape'] != 0:
		average += intensity['Rape']
		count += 1
		flag = 0
	intensity['DomesticViolence'] = gridDV[int(round(x))][int(round(y))]
	if intensity['DomesticViolence'] != 0:
		average += intensity['DomesticViolence']
		count += 1
		flag = 0
	intensity['Theft'] = gridT[int(round(x))][int(round(y))]
	if intensity['Theft'] != 0:
		average += intensity['Theft']
		count += 1
		flag = 0
	intensity['Fraud'] = gridF[int(round(x))][int(round(y))]
	if intensity['Fraud'] != 0:
		average += intensity['Fraud']
		count += 1
	intensity['Drugs'] = gridD[int(round(x))][int(round(y))]
	if intensity['Drugs'] != 0:
		average += intensity['Drugs']
		count += 1
		flag = 0
	TOC = max(intensity.items(), key=operator.itemgetter(1))[0]
	if flag == 1:
		average = 0
	else:
		average /= count
	# severity = intensity[TOC]
	severity = average
	return severity

FileNotFoundError: ignored

In [0]:
Intensity = getIntensity(lat,long)

NameError: ignored

In [0]:
#Model Constants

intercept = 0.928568881
timeSlope = 0.247609499
trafficSlope =	-0.004666748
policeSlope =	0.102761641
populationSlope =	0.207043419
intensitySlope	 = 0.009338735

In [0]:
Severity = (timeSlope * Time) + (trafficSlope * Traffic) + (policeSlope * PoliceStations) +  (populationSlope * Population) + (intensitySlope * Intensity) + intercept

NameError: ignored

In [0]:
#Result of the code
print('Intensity: ',Intensity)
print('Time: ',Time)
print('Population :',Population)
print('Traffic :',Traffic)
print('Police Stations :',PoliceStations)
print('Severity :',Severity)

In [0]:
#Everything in a single function
def multipleLinearRegression(lat,long,time):
  latLong = []
  for i,row in masterTable.iterrows():
    latLong += [(row[5],row[6])]
  tree = spatial.KDTree(latLong)
  dist = tree.query([(lat,long)])[0][0]
  index = tree.query([(lat,long)])[1][0]
  PoliceStations = masterTable['PoliceStations'][index]
  Population = masterTable['c-population'][index]
  Traffic = masterTable['c-traffic'][index]
  Time = convertTimetoCategoricalValue(time)
  Intensity = getIntensity(lat,long)
  Severity = (timeSlope * Time) + (trafficSlope * Traffic) + (policeSlope * PoliceStations) +  (populationSlope * Population) + (intensitySlope * Intensity) + intercept
  return Severity